In [ ]:
!pip install -U heamy &> /dev/null
!pip install git+https://github.com/pfnet-research/xfeat.git &> /dev/null
!pip install catboost &> /dev/null

In [1]:
import lightgbm as lgb
import japanize_matplotlib
import pandas as pd
import numpy as np
import seaborn as sns
import optuna
import pickle
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from catboost import Pool
from catboost import CatBoostRegressor

from heamy.dataset import Dataset
from heamy.estimator import Regressor
from heamy.pipeline import ModelsPipeline

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from modules.customcatboost import CustomCatBoostRegressor
from modules.customlightgbm import CustomLightGBM

In [2]:
df = pickle.load(open('fixed/df_std.sav', 'rb'))

In [3]:
ID = 'ID'
TARGET = '取引価格（総額）_log'
dropcol = ['市区町村コード', '取引時点int']
dropval = 0.9
seed = 42
df = df.drop(dropcol, axis=1)
# df.set_index('ID', inplace = True)

cols = df.columns
cols = cols[cols != '取引価格（総額）_log']
cols = cols[cols != '学習データ']
cols = cols[cols != '価格/面積log']
cols = cols[cols != '面積（㎡）']
# cols = cols[cols != 'ID']

test = df[df['学習データ']==False]
train = df[df['学習データ']==True]
best_params_cat = {'l2_leaf_reg': 33.51917340457483, 'random_strength': 0.16779085114202497, 'subsample': 0.9828709713363581, 'objective': 'MAE', 'colsample_bylevel': 0.09, 'depth': 12, 'boosting_type': 'Plain', 'bootstrap_type': 'Bernoulli', 'eval_metric': 'MAE', 'learning_rate': 0.1, 'early_stopping_rounds': 50, 'iterations': 20000, 'verbose': 500, 'loss_function': 'MAE', 'random_seed': 42}


In [ ]:
# dropval = 1
# train_1 = train[(train['価格/面積log'] <= dropval) & (train['価格/面積log'] >= -1*dropval)]
# print(f'train:{len(train)}, train_1:{len(train_1)}')
# print(f"max:{train_1['価格/面積log'].max()}, min:{train_1['価格/面積log'].min()}")

In [ ]:
# X_np_1 = np.array(train_1[cols])
# y_np_1 = train_1['取引価格（総額）_log'].values
# X_submit = np.array(test[cols])
# X_np = np.array(train[cols])
# y_np = train['取引価格（総額）_log'].values

In [4]:
# Datasetの作成 all, 1, 0.9の3種類を作成
dataset_0 = Dataset(train[cols],
                  train[TARGET],
                  test[cols])
dropval = 1
dataset_1 = Dataset(train[(train['価格/面積log'] <= dropval) & (train['価格/面積log'] >= -1*dropval)][cols],
                  train[(train['価格/面積log'] <= dropval) & (train['価格/面積log'] >= -1*dropval)][TARGET],
                  test[cols])
# dropval = 0.9
# dataset_2 = Dataset(train[(train['価格/面積log'] <= dropval) & (train['価格/面積log'] >= -1*dropval)][cols],
#                   train[(train['価格/面積log'] <= dropval) & (train['価格/面積log'] >= -1*dropval)][TARGET],
#                   test[cols])

In [5]:
models = [
    Regressor(dataset=dataset_0, estimator=CustomLightGBM, parameters=None, name='lgb0'),
    Regressor(dataset=dataset_0, estimator=CustomCatBoostRegressor, parameters=None, name='cat0'),
    Regressor(dataset=dataset_0, estimator=LinearRegression, parameters={'normalize': True}, name='lr0'),
    Regressor(dataset=dataset_1, estimator=CustomLightGBM, parameters=None, name='lgb'),
    Regressor(dataset=dataset_1, estimator=CustomCatBoostRegressor, parameters=None, name='cat'),
    Regressor(dataset=dataset_1, estimator=LinearRegression, parameters={'normalize': True}, name='lr'),
    # Regressor(dataset=dataset_2, estimator=CustomLightGBM, parameters=None, name='lgb2'),
    # Regressor(dataset=dataset_2, estimator=CustomCatBoostRegressor, parameters=None, name='cat2'),
    # Regressor(dataset=dataset_2, estimator=LinearRegression, parameters={'normalize': True}, name='lr2'),
]

In [6]:
# pipelineを定義、2nd levelデータセットの作成
pipeline = ModelsPipeline(*models)

In [7]:
%%time
stack_ds = pipeline.stack(k=10, seed=seed)

学習開始：LightGBM
[500]	Train's l1: 0.0708521	Test's l1: 0.0784319
[1000]	Train's l1: 0.0653239	Test's l1: 0.0775463
[1500]	Train's l1: 0.0621299	Test's l1: 0.0772857
Score: 0.0771630512367004
学習終了：LightGBM
学習開始：LightGBM
[500]	Train's l1: 0.0712668	Test's l1: 0.078279
[1000]	Train's l1: 0.0656047	Test's l1: 0.077207
[1500]	Train's l1: 0.0624265	Test's l1: 0.0769405
[2000]	Train's l1: 0.0606121	Test's l1: 0.0767703
Score: 0.0767553623328164
学習終了：LightGBM
学習開始：LightGBM
[500]	Train's l1: 0.0710699	Test's l1: 0.0787426
[1000]	Train's l1: 0.0655311	Test's l1: 0.0779265
Score: 0.07789568732193872
学習終了：LightGBM
学習開始：LightGBM
[500]	Train's l1: 0.0709517	Test's l1: 0.07903
[1000]	Train's l1: 0.0654025	Test's l1: 0.0781593
Score: 0.07812626333759748
学習終了：LightGBM
学習開始：LightGBM
[500]	Train's l1: 0.0711369	Test's l1: 0.0783979
[1000]	Train's l1: 0.0655714	Test's l1: 0.0774851
[1500]	Train's l1: 0.0623765	Test's l1: 0.0772081
Score: 0.07714143088119611
学習終了：LightGBM
学習開始：LightGBM
[500]	Train's l1: 0.07

In [8]:
X_train_0 = stack_ds.X_train[['lgb0','cat0','lr0']]
X_train_1 = stack_ds.X_train[['lgb','cat','lr']].dropna()
# X_train_2 = stack_ds.X_train[['lgb2','cat2','lr2']].dropna()

In [9]:
X_df = dataset_0.X_train.reset_index().drop('index', axis=1)
X_df[X_train_0.columns] = X_train_0[X_train_0.columns]

In [10]:
X_df_1 = dataset_1.X_train.reset_index().drop('index', axis=1)[[ID]]
X_df_1[X_train_1.columns] = X_train_1[X_train_1.columns]

In [ ]:
# X_df_2 = dataset_2.X_train.reset_index().drop('index', axis=1)[[ID]]
# X_df_2[X_train_2.columns] = X_train_2[X_train_2.columns]

In [14]:
X_df.head(2)

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,建築年,建物の構造,用途,...,agg_<function max_min at 0x7fb31d032ee0>_取引時点築年数_grpby_取引の事情等,agg_<function max_min at 0x7fb31d032ee0>_面積log_grpby_取引の事情等,agg_<function max_min at 0x7fb31d032ee0>_乗降客数2019_grpby_取引の事情等,面積log_scaled,lgb0,cat0,lr0,lgb,cat,lr
0,1012904,0,0,0,0,-0.626784,0,0.106233,0,0,...,0.045476,0.131239,0.014664,-0.510117,6.799807,6.977318,6.799927,6.922514,6.938469,6.806564
1,1014984,0,0,1,1,-0.298684,1,-0.745521,0,0,...,0.045476,0.131239,0.014664,1.245958,7.011446,7.037549,7.072571,7.020739,7.009767,7.079224


In [11]:
X_df = pd.merge(X_df, X_df_1, on=ID)

In [ ]:
# X_df = pd.merge(X_df, X_df_2, on=ID)

In [12]:
X_test_df = dataset_1.X_test.reset_index().drop('index', axis=1)
X_test_pred_df = stack_ds.X_test
X_test_df[X_test_pred_df.columns] = X_test_pred_df[X_test_pred_df.columns]

In [13]:
X_test_df.head(2)

,ID,都道府県名,市区町村名,地区名,最寄駅：名称,最寄駅：距離（分）,間取り,建築年,建物の構造,用途,...,agg_<function max_min at 0x7fb31d032ee0>_取引時点築年数_grpby_取引の事情等,agg_<function max_min at 0x7fb31d032ee0>_面積log_grpby_取引の事情等,agg_<function max_min at 0x7fb31d032ee0>_乗降客数2019_grpby_取引の事情等,面積log_scaled,lgb0,cat0,lr0,lgb,cat,lr
0,1108303,0,9,314,80,-0.462734,17,0.617286,0,0,...,0.045476,0.131239,0.014664,1.044034,7.262801,7.190848,7.264252,7.291137,7.302836,7.265076
1,1108442,0,9,315,80,-0.626784,9,0.702462,0,1,...,0.045476,0.131239,0.014664,1.044034,7.269189,7.279555,7.278320,7.262439,7.227407,7.278870


In [15]:
dataset_stack = Dataset(X_df, dataset_1.y_train, X_test_df)

In [16]:
%%time
# modelを作ってvalidation
stacker = Regressor(dataset=dataset_stack, estimator=CustomCatBoostRegressor)
y_trues, y_preds = stacker.validate(k=1)
# 精度出力
cv_results = []
for y_true, y_pred in zip(y_trues, y_preds):
    cv_result = mean_absolute_error(y_true, y_pred)
    cv_results.append(cv_result)
print(f'stacking: {np.mean(cv_results):.4f}')

学習開始：CatBoost
0:	learn: 0.2421909	test: 0.2423863	best: 0.2423863 (0)	total: 350ms	remaining: 1h 56m 41s
Stopped by overfitting detector  (50 iterations wait)

bestTest = 0.06765888063
bestIteration = 239

Shrink model to first 240 iterations.
学習終了：CatBoost
stacking: 0.0676
CPU times: user 11min 1s, sys: 7.9 s, total: 11min 9s
Wall time: 1min 44s


In [ ]:
X_test_df

In [20]:
# モデルを保存する
filename = 'dataset/stack.sav'
pickle.dump(dataset_stack, open(filename, 'wb'))

In [ ]:
# dataset_1のみ
# CatBoost 0.0689 →0.0731
# dataset_0,1両方 (0.0681)
# Catboost 0.0685 →0.0723
# dataset_0,1両方k10 (0.0676)
# Catboost 0.067? →0.0
# dataset_0,1,2 (0.0666)
# Catboost 0.0670 →0.0726

In [ ]:
# FOLD = 10
# valid_scores_cat = []
# models_cat = []
# kf = KFold(n_splits=FOLD, shuffle=True, random_state=42)
# for fold, (train_indices, valid_indices) in enumerate(kf.split(dataset_stack.X_train.values)):
#     X_train, X_valid = dataset_stack.X_train.values[train_indices], dataset_stack.X_train.values[valid_indices]
#     y_train, y_valid = dataset_stack.y_train[train_indices], dataset_stack.y_train[valid_indices]
#     train_pool = Pool(X_train, y_train)
#     validate_pool = Pool(X_valid, y_valid)
    
#     model = CatBoostRegressor(**best_params_cat)
#     model.fit(train_pool, eval_set=validate_pool)

#     y_valid_pred = model.predict(X_valid)
    
#     score = mean_absolute_error(y_valid, y_valid_pred)
#     print(f'fold {fold} MAE: {score}')
#     valid_scores_cat.append(score)

#     models_cat.append(model)

# cv_score = np.mean(valid_scores_cat)
# print(f'CV score: {cv_score}')

In [ ]:
# submit_pred_cats = []
# for i in range(FOLD):
#     submit_pred = models_cat[i].predict(X_test_df.values)
#     submit_pred_cats.append(submit_pred)

# for i in range(FOLD):
#     if i == 0:
#         submit_pred_cat = submit_pred_cats[0]
#     else:
#         submit_pred_cat = submit_pred_cat + submit_pred_cats[i]

# submit_pred_cat = submit_pred_cat/FOLD
# submit_pred_cat = np.log10(np.ceil(np.power(10, submit_pred_cat)/100000)*100000)

In [ ]:
# X_submit_df = X_test_df
# submit_pred_cat_df = pd.DataFrame(submit_pred_cat, columns=[TARGET])
# submit_pred_cat_df[ID] = X_submit_df[ID]
# submit_pred_cat_df = submit_pred_cat_df[[ID, TARGET]].astype({ID:int})
# # submit_pred_cat_df.to_csv('./output/submission_cat.csv', index = False)

In [ ]:
# submit_pred_cat

In [ ]:
# sub_df = pd.read_csv('data/sample_submission.csv')
# sub_df = pd.merge(sub_df[['ID']], submit_pred_cat_df[['ID', TARGET]], on='ID')
# sub_df.to_csv('output/submission_cat012.csv', index=False)

In [ ]:
# CatBoost, CustomLightGBM
# CatBoost                   0.0697
# CustomLightGBM             0.0696
# LinearRegression           0.0694
# RandomForestRegressor      0.0768

# CatBoost, CustomLightGBM, LinearRegression
# CatBoost                   0.0697
# CustomLightGBM             0.0696
# LinearRegression           0.0694
# RandomForestRegressor      0.0768


In [ ]:
# X_testを使ってpredict
y_pred = stacker.predict()

In [ ]:
submit_pred_cat = np.log10(np.ceil(np.power(10, y_pred)/100000)*100000)

In [ ]:
X_submit_df = pd.DataFrame(test[ID], columns=[ID])

In [ ]:
X_submit_df[TARGET] = submit_pred_cat

In [ ]:
X_submit_df = X_submit_df.reset_index().drop('index', axis=1)

In [ ]:
sub_df = pd.read_csv('data/sample_submission.csv')
sub_df = pd.merge(sub_df[[ID]], X_submit_df[[ID, TARGET]], on=ID)

In [ ]:
sub_df.to_csv('output/test_submission.csv', index=False)

In [ ]:

# datasetを準備
# dataset = Dataset(train_1[cols], train_1['取引価格（総額）_log'], test[cols]) # X_testは今回使わないが入れないとエラーになる


# アンサンブルに使うモデルを定義
# models = [
#     Regressor(dataset=dataset, estimator=RandomForestRegressor, parameters={'n_estimators': 50, 'random_state': seed}, name='rf'),
#     Regressor(dataset=dataset, estimator=LinearRegression, parameters={'normalize': True}, name='lr'),
#     Regressor(dataset=dataset, estimator=KNeighborsRegressor, name='kr'),
#     Regressor(dataset=dataset, estimator=CatBoostRegressor, parameters={'custom_metric': ['MAE'], 'random_seed': seed, 'logging_level': 'Silent'}, name='cr')
# ]

# pipelineを定義、2nd levelデータセットの作成
# pipeline = ModelsPipeline(*models)
# stack_ds = pipeline.stack(k=10, seed=seed)

# modelを作ってvalidation
# stacker = Regressor(dataset=stack_ds, estimator=LinearRegression)
# y_trues, y_preds = stacker.validate(k=10)

# 精度出力
# cv_results = []
# for y_true, y_pred in zip(y_trues, y_preds):
#     cv_result = mean_absolute_error(y_true, y_pred)
#     cv_results.append(cv_result)
# print(f'stacking: {np.mean(cv_results):.2f}')

# X_testを使ってpredict
# y_pred = stacker.predict()